In [2]:
#%pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124 --force-reinstall

In [3]:
"""import torch

def format_pytorch_version(version_str):
    # Example input: "2.0.1+cu118" -> returns "2.0.1"
    return version_str.split('+')[0]

def format_cuda_version(cuda_str):
    # If CUDA is None (CPU-only PyTorch), return "cpu"
    if cuda_str is None:
        return "cpu"
    # Example: "11.8" -> "cu118"
    return "cu" + cuda_str.replace('.', '')

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)
CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

print(f"Installing packages for torch-{TORCH}+{CUDA}...")

%pip install --upgrade --no-cache-dir pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html

%pip install torch_geometric"""

'import torch\n\ndef format_pytorch_version(version_str):\n    # Example input: "2.0.1+cu118" -> returns "2.0.1"\n    return version_str.split(\'+\')[0]\n\ndef format_cuda_version(cuda_str):\n    # If CUDA is None (CPU-only PyTorch), return "cpu"\n    if cuda_str is None:\n        return "cpu"\n    # Example: "11.8" -> "cu118"\n    return "cu" + cuda_str.replace(\'.\', \'\')\n\nTORCH_version = torch.__version__\nTORCH = format_pytorch_version(TORCH_version)\nCUDA_version = torch.version.cuda\nCUDA = format_cuda_version(CUDA_version)\n\nprint(f"Installing packages for torch-{TORCH}+{CUDA}...")\n\n%pip install --upgrade --no-cache-dir pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html\n\n%pip install torch_geometric'

In [4]:
#%pip install scikit-learn imbalanced-learn xgboost

In [5]:
#%pip install optuna

### Hyperparameter Optimization Citeseer
#### SCAR 

In [2]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "citeseer",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 2, 7),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.09, 0.18),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs": trial.suggest_int("layers", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [256,512,1024]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "citeseer_scar_sampling_nnif_batch_cluster.csv",
        "min":0.845,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,         
        "sampling":trial.suggest_categorical("sampling",["neighbor","cluster","nearest_neighbors"]),         
        "sampling_k":trial.suggest_categorical("sampling_k",[25,50,75]),
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-26 22:01:54,897] A new study created in memory with name: no-name-37ad9fee-880f-4bf7-9590-7b67e2bc2290


Running experiment with seed=654:
 - K=7, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=256, methodology=ours
 - ratio=0.13932336327408168, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=nearest_neighbors, sampling_k=75
 - model_type=GCNConv, rate_pairs=1, clusters=100, lr=0.003087514626074824
Epoch 0 / 100, Loss: 27.7274
Epoch 10 / 100, Loss: 21.4100
Epoch 20 / 100, Loss: 16.7578
Epoch 30 / 100, Loss: 12.9332
Epoch 40 / 100, Loss: 10.0351
Epoch 50 / 100, Loss: 7.9643
Epoch 60 / 100, Loss: 6.3783
Epoch 70 / 100, Loss: 5.2735
Epoch 80 / 100, Loss: 4.3328
Epoch 90 / 100, Loss: 3.6551
 - Metrics: Accuracy=0.9369, F1=0.8456, Recall=0.8203, Precision=0.8725
Running experiment with seed=114:
 - K=7, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=256, methodology=ours
 - ratio=0.13932336327408168, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=nearest_neighbors, sampling_k=75
 - model_

[I 2025-03-26 22:08:54,753] Trial 0 finished with value: 0.8453960694225009 and parameters: {'K': 7, 'layers': 1, 'hidden_channels': 256, 'out_channels': 256, 'ratio': 0.13932336327408168, 'model_type': 'GCNConv', 'batch_size': 256, 'lr': 0.003087514626074824, 'clusters': 100, 'sampling': 'nearest_neighbors', 'sampling_k': 75}. Best is trial 0 with value: 0.8453960694225009.


 - Metrics: Accuracy=0.9324, F1=0.8356, Recall=0.8160, Precision=0.8563
F1 = 0.84 < 0.845, skipping ...
Done. Results written to citeseer_experimentations\citeseer_scar_sampling_nnif_batch_cluster_2603220154.csv.
Average F1 over valid seeds: 0.8454 ± 0.0063
Running experiment with seed=654:
 - K=2, layers=2, hidden=256, out=128
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.13750087548779888, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=75
 - model_type=GATConv, rate_pairs=2, clusters=300, lr=0.0015207494162327235
Epoch 0 / 100, Loss: 2.2770
Epoch 10 / 100, Loss: 2.0525
Epoch 20 / 100, Loss: 2.0233
Epoch 30 / 100, Loss: 1.8897
Epoch 40 / 100, Loss: 1.8259
Epoch 50 / 100, Loss: 1.7592
Epoch 60 / 100, Loss: 1.7085
Epoch 70 / 100, Loss: 1.6272
Epoch 80 / 100, Loss: 1.5657
Epoch 90 / 100, Loss: 1.5083


[I 2025-03-26 22:09:32,505] Trial 1 finished with value: 0.8216216216216217 and parameters: {'K': 2, 'layers': 2, 'hidden_channels': 256, 'out_channels': 128, 'ratio': 0.13750087548779888, 'model_type': 'GATConv', 'batch_size': 1024, 'lr': 0.0015207494162327235, 'clusters': 300, 'sampling': 'neighbor', 'sampling_k': 75}. Best is trial 0 with value: 0.8453960694225009.


...

 - Metrics: Accuracy=0.9420, F1=0.8571, Recall=0.8260, Precision=0.8908
Done. Results written to citeseer_experimentations\citeseer_scar_sampling_nnif_batch_cluster_2703090009.csv.
Average F1 over valid seeds: 0.8614 ± 0.0073
Running experiment with seed=654:
 - K=3, layers=1, hidden=64, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.11266072511121758, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=75
 - model_type=GATConv, rate_pairs=1, clusters=400, lr=0.007405345109839189
Epoch 0 / 100, Loss: 3.0388
Epoch 10 / 100, Loss: 2.4135
Epoch 20 / 100, Loss: 1.9736
Epoch 30 / 100, Loss: 1.6323
Epoch 40 / 100, Loss: 1.2881
Epoch 50 / 100, Loss: 1.0654
Epoch 60 / 100, Loss: 0.8707
Epoch 70 / 100, Loss: 0.7608
Epoch 80 / 100, Loss: 0.6406
Epoch 90 / 100, Loss: 0.5536
 - Metrics: Accuracy=0.9519, F1=0.8815, Recall=0.8488, Precision=0.9168
Running experiment with seed=114:
 - K=3, layers=1, hidden=64, out=256
 

[I 2025-03-27 09:05:47,916] Trial 198 finished with value: 0.8702456042121606 and parameters: {'K': 3, 'layers': 1, 'hidden_channels': 64, 'out_channels': 256, 'ratio': 0.11266072511121758, 'model_type': 'GATConv', 'batch_size': 1024, 'lr': 0.007405345109839189, 'clusters': 400, 'sampling': 'neighbor', 'sampling_k': 75}. Best is trial 156 with value: 0.8722422037353572.


 - Metrics: Accuracy=0.9468, F1=0.8694, Recall=0.8402, Precision=0.9006
Done. Results written to citeseer_experimentations\citeseer_scar_sampling_nnif_batch_cluster_2703090241.csv.
Average F1 over valid seeds: 0.8702 ± 0.0059
Running experiment with seed=654:
 - K=3, layers=1, hidden=64, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.11442266685450465, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=75
 - model_type=GATConv, rate_pairs=1, clusters=400, lr=0.0074238320254282454
Epoch 0 / 100, Loss: 3.0382
Epoch 10 / 100, Loss: 2.4131
Epoch 20 / 100, Loss: 1.9727
Epoch 30 / 100, Loss: 1.6299
Epoch 40 / 100, Loss: 1.2857
Epoch 50 / 100, Loss: 1.0629
Epoch 60 / 100, Loss: 0.8686
Epoch 70 / 100, Loss: 0.7585
Epoch 80 / 100, Loss: 0.6390
Epoch 90 / 100, Loss: 0.5519
 - Metrics: Accuracy=0.9498, F1=0.8773, Recall=0.8516, Precision=0.9045
Running experiment with seed=114:
 - K=3, layers=1, hidden=64, out=256


[I 2025-03-27 09:08:56,552] Trial 199 finished with value: 0.8693388634016216 and parameters: {'K': 3, 'layers': 1, 'hidden_channels': 64, 'out_channels': 256, 'ratio': 0.11442266685450465, 'model_type': 'GATConv', 'batch_size': 1024, 'lr': 0.0074238320254282454, 'clusters': 400, 'sampling': 'neighbor', 'sampling_k': 75}. Best is trial 156 with value: 0.8722422037353572.


 - Metrics: Accuracy=0.9444, F1=0.8631, Recall=0.8317, Precision=0.8969
Done. Results written to citeseer_experimentations\citeseer_scar_sampling_nnif_batch_cluster_2703090547.csv.
Average F1 over valid seeds: 0.8693 ± 0.0055
Best trial:
  Average F1: 0.8722422037353572
  Best parameters:
    K: 3
    layers: 1
    hidden_channels: 128
    out_channels: 256
    ratio: 0.14573085963762972
    model_type: GATConv
    batch_size: 1024
    lr: 0.006079262879388443
    clusters: 100
    sampling: neighbor
    sampling_k: 75


### Hyperparameter Optimization Citeseer
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "citeseer",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 2, 7),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.09, 0.18),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("layers", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [256,512,1024]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "citeseer_sar_sampling_nnif_batch_cluster.csv",
        "min":0.845,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),
        "num_epochs":100,         
        "sampling":trial.suggest_categorical("sampling",["neighbor","cluster","nearest_neighbors"]),         
        "sampling_k":trial.suggest_categorical("sampling_k",[25,50,75]),
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-25 10:10:43,640] A new study created in memory with name: no-name-f25fde7d-ee4a-4fa2-be4f-f9e486b8970e


Running experiment with seed=654:
 - K=3, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.11307195638376069, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=75
 - model_type=GATConv, rate_pairs=1, clusters=50, lr=0.0023234250431828636
Epoch 0 / 100, Loss: 2.8385
Epoch 10 / 100, Loss: 2.4108
Epoch 20 / 100, Loss: 2.2628
Epoch 30 / 100, Loss: 2.1825
Epoch 40 / 100, Loss: 2.0409
Epoch 50 / 100, Loss: 1.9945
Epoch 60 / 100, Loss: 1.8232
Epoch 70 / 100, Loss: 1.6971
Epoch 80 / 100, Loss: 1.6468
Epoch 90 / 100, Loss: 1.4848
 - Metrics: Accuracy=0.9405, F1=0.8507, Recall=0.8046, Precision=0.9024
Running experiment with seed=114:
 - K=3, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.11307195638376069, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=75
 - model_type=GATConv, rate_pa

[I 2025-03-25 10:13:49,885] Trial 0 finished with value: 0.857595508989174 and parameters: {'K': 3, 'layers': 1, 'hidden_channels': 256, 'out_channels': 256, 'ratio': 0.11307195638376069, 'model_type': 'GATConv', 'batch_size': 1024, 'lr': 0.0023234250431828636, 'anomaly_detector': 'nearest_neighbors', 'clusters': 50, 'num_epochs': 100, 'sampling': 'neighbor', 'sampling_k': 75}. Best is trial 0 with value: 0.857595508989174.


 - Metrics: Accuracy=0.9390, F1=0.8493, Recall=0.8160, Precision=0.8854
Done. Results written to citeseer_experimentations\citeseer_sar_sampling_nnif_batch_cluster_2503101043.csv.
Average F1 over valid seeds: 0.8576 ± 0.0077
Running experiment with seed=654:
 - K=7, layers=2, hidden=256, out=64
 - norm=None, dropout=0, batch_size=256, methodology=ours
 - ratio=0.17225386531043968, aggregation=mean, treatment=removal, anomaly_detector=unweighted, sampling=nearest_neighbors, sampling_k=25
 - model_type=GATConv, rate_pairs=2, clusters=400, lr=0.008752505664905935
Epoch 0 / 150, Loss: 42.4167
Epoch 10 / 150, Loss: 18.6557
Epoch 20 / 150, Loss: 10.0401
Epoch 30 / 150, Loss: 6.0609
Epoch 40 / 150, Loss: 4.2251
Epoch 50 / 150, Loss: 3.0423
Epoch 60 / 150, Loss: 2.2833
Epoch 70 / 150, Loss: 1.8259
Epoch 80 / 150, Loss: 1.4801
Epoch 90 / 150, Loss: 1.2157
Epoch 100 / 150, Loss: 1.0309
Epoch 110 / 150, Loss: 0.8732
Epoch 120 / 150, Loss: 0.7590
Epoch 130 / 150, Loss: 0.6525
Epoch 140 / 150, Loss

[I 2025-03-25 10:15:40,677] Trial 1 finished with value: 0.6813996316758748 and parameters: {'K': 7, 'layers': 2, 'hidden_channels': 256, 'out_channels': 64, 'ratio': 0.17225386531043968, 'model_type': 'GATConv', 'batch_size': 256, 'lr': 0.008752505664905935, 'anomaly_detector': 'unweighted', 'clusters': 400, 'num_epochs': 150, 'sampling': 'nearest_neighbors', 'sampling_k': 25}. Best is trial 0 with value: 0.857595508989174.


...

 - Metrics: Accuracy=0.9059, F1=0.7183, Recall=0.5692, Precision=0.9732
F1 = 0.72 < 0.845, skipping ...
Done. Results written to citeseer_experimentations\citeseer_sar_sampling_nnif_batch_cluster_2503140907.csv.
Average F1 over valid seeds: 0.7183 ± 0.0000
Running experiment with seed=654:
 - K=2, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.15841141409782936, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=25
 - model_type=GCNConv, rate_pairs=1, clusters=400, lr=0.004733199465670689
Epoch 0 / 100, Loss: 2.1699
Epoch 10 / 100, Loss: 1.6532
Epoch 20 / 100, Loss: 1.4729
Epoch 30 / 100, Loss: 1.3237
Epoch 40 / 100, Loss: 1.1338
Epoch 50 / 100, Loss: 1.0336
Epoch 60 / 100, Loss: 0.8867
Epoch 70 / 100, Loss: 0.7703
Epoch 80 / 100, Loss: 0.7087
Epoch 90 / 100, Loss: 0.6093
 - Metrics: Accuracy=0.9408, F1=0.8600, Recall=0.8631, Precision=0.8569
Running experiment with seed=114:
 - K=2

[I 2025-03-25 14:12:40,226] Trial 98 finished with value: 0.8617602498928539 and parameters: {'K': 2, 'layers': 1, 'hidden_channels': 256, 'out_channels': 256, 'ratio': 0.15841141409782936, 'model_type': 'GCNConv', 'batch_size': 1024, 'lr': 0.004733199465670689, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 100, 'sampling': 'neighbor', 'sampling_k': 25}. Best is trial 87 with value: 0.8650138572197479.


 - Metrics: Accuracy=0.9387, F1=0.8539, Recall=0.8502, Precision=0.8576
Done. Results written to citeseer_experimentations\citeseer_sar_sampling_nnif_batch_cluster_2503140941.csv.
Average F1 over valid seeds: 0.8618 ± 0.0066
Running experiment with seed=654:
 - K=2, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.16102039029803536, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=nearest_neighbors, sampling_k=25
 - model_type=GCNConv, rate_pairs=1, clusters=400, lr=0.004605584968207975
Epoch 0 / 100, Loss: 4.1609
Epoch 10 / 100, Loss: 3.4473
Epoch 20 / 100, Loss: 3.0725
Epoch 30 / 100, Loss: 2.6641
Epoch 40 / 100, Loss: 2.3129
Epoch 50 / 100, Loss: 2.1049
Epoch 60 / 100, Loss: 1.7786
Epoch 70 / 100, Loss: 1.6623
Epoch 80 / 100, Loss: 1.4632
Epoch 90 / 100, Loss: 1.3323


[I 2025-03-25 14:13:15,258] Trial 99 finished with value: 0.8429629629629629 and parameters: {'K': 2, 'layers': 1, 'hidden_channels': 256, 'out_channels': 256, 'ratio': 0.16102039029803536, 'model_type': 'GCNConv', 'batch_size': 1024, 'lr': 0.004605584968207975, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 100, 'sampling': 'nearest_neighbors', 'sampling_k': 25}. Best is trial 87 with value: 0.8650138572197479.


 - Metrics: Accuracy=0.9363, F1=0.8430, Recall=0.8117, Precision=0.8767
F1 = 0.84 < 0.845, skipping ...
Done. Results written to citeseer_experimentations\citeseer_sar_sampling_nnif_batch_cluster_2503141240.csv.
Average F1 over valid seeds: 0.8430 ± 0.0000
Best trial:
  Average F1: 0.8650138572197479
  Best parameters:
    K: 2
    layers: 1
    hidden_channels: 128
    out_channels: 256
    ratio: 0.1611386183555754
    model_type: GCNConv
    batch_size: 1024
    lr: 0.0063212510924867535
    anomaly_detector: nearest_neighbors
    clusters: 200
    num_epochs: 100
    sampling: neighbor
    sampling_k: 25


### Hyperparameter Optimization Cora
#### SCAR 

In [4]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "cora",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 1, 12),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.16, 0.34),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("layers", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [15,256,512,1024]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "cora_scar_sampling_nnif_batch_cluster.csv",
        "min":0.85,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,         
        "sampling":trial.suggest_categorical("sampling",["neighbor","cluster","nearest_neighbors"]),         
        "sampling_k":trial.suggest_categorical("sampling_k",[25,50,75]),
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-27 10:52:51,733] A new study created in memory with name: no-name-0e26df09-7fa5-46d1-be19-f32a53fd7c0d


Running experiment with seed=654:
 - K=10, layers=1, hidden=64, out=64
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.19622450292840055, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=50
 - model_type=GCNConv, rate_pairs=1, clusters=400, lr=0.003767773886669508
Epoch 0 / 100, Loss: 8.2044
Epoch 10 / 100, Loss: 7.3760
Epoch 20 / 100, Loss: 6.8701
Epoch 30 / 100, Loss: 6.4718
Epoch 40 / 100, Loss: 5.6112
Epoch 50 / 100, Loss: 4.9094
Epoch 60 / 100, Loss: 4.1198
Epoch 70 / 100, Loss: 3.4837
Epoch 80 / 100, Loss: 2.9254
Epoch 90 / 100, Loss: 2.5098
 - Metrics: Accuracy=0.9232, F1=0.8706, Recall=0.8557, Precision=0.8861
Running experiment with seed=114:
 - K=10, layers=1, hidden=64, out=64
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.19622450292840055, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=50
 - model_type=GCNConv, rate_pairs=

[I 2025-03-27 10:55:58,397] Trial 0 finished with value: 0.8818876658327218 and parameters: {'K': 10, 'layers': 1, 'hidden_channels': 64, 'out_channels': 64, 'ratio': 0.19622450292840055, 'model_type': 'GCNConv', 'batch_size': 512, 'lr': 0.003767773886669508, 'clusters': 400, 'sampling': 'neighbor', 'sampling_k': 50}. Best is trial 0 with value: 0.8818876658327218.


 - Metrics: Accuracy=0.9343, F1=0.8882, Recall=0.8643, Precision=0.9134
Done. Results written to cora_experimentations\cora_scar_sampling_nnif_batch_cluster_2703105251.csv.
Average F1 over valid seeds: 0.8819 ± 0.0069
Running experiment with seed=654:
 - K=8, layers=1, hidden=128, out=256
 - norm=None, dropout=0, batch_size=15, methodology=ours
 - ratio=0.22715757417672017, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=nearest_neighbors, sampling_k=25
 - model_type=GCNConv, rate_pairs=1, clusters=200, lr=0.008027462923101808
Epoch 0 / 100, Loss: 443.7636
Epoch 10 / 100, Loss: 9.9173
Epoch 20 / 100, Loss: 2.5276
Epoch 30 / 100, Loss: 0.8766
Epoch 40 / 100, Loss: 0.3332
Epoch 50 / 100, Loss: 0.1330
Epoch 60 / 100, Loss: 0.0536
Epoch 70 / 100, Loss: 0.0216
Epoch 80 / 100, Loss: 0.0088
Epoch 90 / 100, Loss: 0.0035


[I 2025-03-27 11:05:52,536] Trial 1 finished with value: 0.7062359128474831 and parameters: {'K': 8, 'layers': 1, 'hidden_channels': 128, 'out_channels': 256, 'ratio': 0.22715757417672017, 'model_type': 'GCNConv', 'batch_size': 15, 'lr': 0.008027462923101808, 'clusters': 200, 'sampling': 'nearest_neighbors', 'sampling_k': 25}. Best is trial 0 with value: 0.8818876658327218.


...

 - Metrics: Accuracy=0.9501, F1=0.9194, Recall=0.9413, Precision=0.8985
Done. Results written to cora_experimentations\cora_scar_sampling_nnif_batch_cluster_2803040318.csv.
Average F1 over valid seeds: 0.9119 ± 0.0067
Running experiment with seed=654:
 - K=4, layers=1, hidden=64, out=128
 - norm=None, dropout=0, batch_size=256, methodology=ours
 - ratio=0.25938262754115543, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=25
 - model_type=GCNConv, rate_pairs=1, clusters=300, lr=0.004480872415563464
Epoch 0 / 100, Loss: 10.6947
Epoch 10 / 100, Loss: 7.8931
Epoch 20 / 100, Loss: 5.6721
Epoch 30 / 100, Loss: 4.1495
Epoch 40 / 100, Loss: 3.0018
Epoch 50 / 100, Loss: 2.2835
Epoch 60 / 100, Loss: 1.7759
Epoch 70 / 100, Loss: 1.4255
Epoch 80 / 100, Loss: 1.1883
Epoch 90 / 100, Loss: 0.9848
 - Metrics: Accuracy=0.9420, F1=0.9053, Recall=0.9169, Precision=0.8939
Running experiment with seed=114:
 - K=4, layers=1, hidden=64, out=128
 - norm=N

[I 2025-03-28 04:12:41,431] Trial 198 finished with value: 0.9035419136951311 and parameters: {'K': 4, 'layers': 1, 'hidden_channels': 64, 'out_channels': 128, 'ratio': 0.25938262754115543, 'model_type': 'GCNConv', 'batch_size': 256, 'lr': 0.004480872415563464, 'clusters': 300, 'sampling': 'neighbor', 'sampling_k': 25}. Best is trial 111 with value: 0.9150370188561521.


 - Metrics: Accuracy=0.9383, F1=0.9012, Recall=0.9315, Precision=0.8729
Done. Results written to cora_experimentations\cora_scar_sampling_nnif_batch_cluster_2803040817.csv.
Average F1 over valid seeds: 0.9035 ± 0.0047
Running experiment with seed=654:
 - K=4, layers=1, hidden=64, out=256
 - norm=None, dropout=0, batch_size=15, methodology=ours
 - ratio=0.254081859193035, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=25
 - model_type=GATConv, rate_pairs=1, clusters=400, lr=0.005256282558691244
Epoch 0 / 100, Loss: 160.8666
Epoch 10 / 100, Loss: 5.6128
Epoch 20 / 100, Loss: 1.4399
Epoch 30 / 100, Loss: 0.4975
Epoch 40 / 100, Loss: 0.1918
Epoch 50 / 100, Loss: 0.0757
Epoch 60 / 100, Loss: 0.0305
Epoch 70 / 100, Loss: 0.0129
Epoch 80 / 100, Loss: 0.0051
Epoch 90 / 100, Loss: 0.0021


[I 2025-03-28 04:20:37,928] Trial 199 finished with value: 0.7777031564808596 and parameters: {'K': 4, 'layers': 1, 'hidden_channels': 64, 'out_channels': 256, 'ratio': 0.254081859193035, 'model_type': 'GATConv', 'batch_size': 15, 'lr': 0.005256282558691244, 'clusters': 400, 'sampling': 'neighbor', 'sampling_k': 25}. Best is trial 111 with value: 0.9150370188561521.


 - Metrics: Accuracy=0.8778, F1=0.7777, Recall=0.7078, Precision=0.8629
F1 = 0.78 < 0.85, skipping ...
Done. Results written to cora_experimentations\cora_scar_sampling_nnif_batch_cluster_2803041241.csv.
Average F1 over valid seeds: 0.7777 ± 0.0000
Best trial:
  Average F1: 0.9150370188561521
  Best parameters:
    K: 4
    layers: 1
    hidden_channels: 128
    out_channels: 256
    ratio: 0.26218826843662685
    model_type: GATConv
    batch_size: 256
    lr: 0.00302395227957548
    clusters: 100
    sampling: neighbor
    sampling_k: 25


### Hyperparameter Optimization Cora
#### SAR 

In [5]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "cora",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 7, 9),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.16, 0.34),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("layers", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [256,512,1024]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "cora_sar_sampling_nnif_batch_cluster.csv",
        "min":0.85,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":trial.suggest_categorical("num_epochs", [50,100,150]),         
        "sampling":trial.suggest_categorical("sampling",["neighbor","cluster","nearest_neighbors"]),         
        "sampling_k":trial.suggest_categorical("sampling_k",[25,50,75]),
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-26 15:51:16,342] A new study created in memory with name: no-name-8176da62-abb1-46e4-8807-5e5ff1459ce0


Running experiment with seed=654:
 - K=9, layers=1, hidden=64, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.16889523261113468, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=75
 - model_type=GCNConv, rate_pairs=1, clusters=400, lr=0.007646163484926425
Epoch 0 / 150, Loss: 3.4303
Epoch 10 / 150, Loss: 3.1501
Epoch 20 / 150, Loss: 2.8808
Epoch 30 / 150, Loss: 2.6330
Epoch 40 / 150, Loss: 2.3545
Epoch 50 / 150, Loss: 2.0481
Epoch 60 / 150, Loss: 1.7211
Epoch 70 / 150, Loss: 1.4545
Epoch 80 / 150, Loss: 1.2300
Epoch 90 / 150, Loss: 1.0373
Epoch 100 / 150, Loss: 0.8788
Epoch 110 / 150, Loss: 0.7530
Epoch 120 / 150, Loss: 0.6579
Epoch 130 / 150, Loss: 0.5790
Epoch 140 / 150, Loss: 0.5190
 - Metrics: Accuracy=0.9228, F1=0.8656, Recall=0.8227, Precision=0.9132
Running experiment with seed=114:
 - K=9, layers=1, hidden=64, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.1688952

[I 2025-03-26 15:53:50,635] Trial 0 finished with value: 0.8684716037566916 and parameters: {'K': 9, 'layers': 1, 'hidden_channels': 64, 'out_channels': 256, 'ratio': 0.16889523261113468, 'model_type': 'GCNConv', 'batch_size': 1024, 'lr': 0.007646163484926425, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 150, 'sampling': 'neighbor', 'sampling_k': 75}. Best is trial 0 with value: 0.8684716037566916.


 - Metrics: Accuracy=0.9269, F1=0.8731, Recall=0.8325, Precision=0.9178
Done. Results written to cora_experimentations\cora_sar_sampling_nnif_batch_cluster_2603155116.csv.
Average F1 over valid seeds: 0.8685 ± 0.0049
Running experiment with seed=654:
 - K=9, layers=1, hidden=128, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.1886114414198443, aggregation=mean, treatment=removal, anomaly_detector=unweighted, sampling=nearest_neighbors, sampling_k=25
 - model_type=GCNConv, rate_pairs=1, clusters=200, lr=0.00016096248213070527
Epoch 0 / 150, Loss: 35.3035
Epoch 10 / 150, Loss: 35.6685
Epoch 20 / 150, Loss: 34.8467
Epoch 30 / 150, Loss: 34.7338
Epoch 40 / 150, Loss: 34.8928
Epoch 50 / 150, Loss: 34.1160
Epoch 60 / 150, Loss: 33.9046
Epoch 70 / 150, Loss: 33.9726
Epoch 80 / 150, Loss: 33.1283
Epoch 90 / 150, Loss: 33.2420
Epoch 100 / 150, Loss: 32.6670
Epoch 110 / 150, Loss: 32.7155
[Early Stopping] at epoch 118


[I 2025-03-26 15:54:30,723] Trial 1 finished with value: 0.6825775656324582 and parameters: {'K': 9, 'layers': 1, 'hidden_channels': 128, 'out_channels': 256, 'ratio': 0.1886114414198443, 'model_type': 'GCNConv', 'batch_size': 512, 'lr': 0.00016096248213070527, 'anomaly_detector': 'unweighted', 'clusters': 200, 'num_epochs': 150, 'sampling': 'nearest_neighbors', 'sampling_k': 25}. Best is trial 0 with value: 0.8684716037566916.


...

Epoch 0 / 100, Loss: 5.9769
Epoch 10 / 100, Loss: 5.1580
Epoch 20 / 100, Loss: 4.5073
Epoch 30 / 100, Loss: 3.6713
Epoch 40 / 100, Loss: 3.1895
Epoch 50 / 100, Loss: 2.3100
Epoch 60 / 100, Loss: 1.8741
Epoch 70 / 100, Loss: 1.5376
Epoch 80 / 100, Loss: 1.2265
Epoch 90 / 100, Loss: 1.0520
 - Metrics: Accuracy=0.9210, F1=0.8694, Recall=0.8704, Precision=0.8683
Running experiment with seed=25:
 - K=8, layers=1, hidden=128, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.2573033125681119, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=cluster, sampling_k=25
 - model_type=GCNConv, rate_pairs=1, clusters=100, lr=0.007397697366787508


Computing METIS partitioning...
Done!


Epoch 0 / 100, Loss: 5.8094


[W 2025-03-26 16:49:22,637] Trial 31 failed with parameters: {'K': 8, 'layers': 1, 'hidden_channels': 128, 'out_channels': 256, 'ratio': 0.2573033125681119, 'model_type': 'GCNConv', 'batch_size': 1024, 'lr': 0.007397697366787508, 'anomaly_detector': 'nearest_neighbors', 'clusters': 100, 'num_epochs': 100, 'sampling': 'cluster', 'sampling_k': 25} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\romai\AppData\Local\Temp\ipykernel_39988\45902056.py", line 39, in objective
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 565, in run_nnif_gnn_experiment
    train_labels, train_proba, train_losses = train_graph(
          

KeyboardInterrupt: 

### Hyperparameter Optimization Pubmed
#### SCAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "pubmed",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 2, 16),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.2, 0.4),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("layers", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [15,256,512,1024]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector": "nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "pubmed_scar_sampling_nnif_batch_cluster.csv",
        "min":0.84,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,         
        "sampling":trial.suggest_categorical("sampling",["neighbor","cluster","nearest_neighbors"]),         
        "sampling_k":trial.suggest_categorical("sampling_k",[25,50,75]),
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-29 19:17:01,393] A new study created in memory with name: no-name-dcd8ff81-1348-4cd8-a2d5-a7b993331727
Computing METIS partitioning...


Running experiment with seed=654:
 - K=9, layers=1, hidden=64, out=128
 - norm=None, dropout=0, batch_size=256, methodology=ours
 - ratio=0.3197108882381444, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=cluster, sampling_k=25
 - model_type=GCNConv, rate_pairs=1, clusters=50, lr=0.0008492447031723229


Done!


Epoch 0 / 100, Loss: 13.3983
Epoch 10 / 100, Loss: 13.1149
Epoch 20 / 100, Loss: 12.6916
Epoch 30 / 100, Loss: 12.6578
Epoch 40 / 100, Loss: 12.2084
Epoch 50 / 100, Loss: 12.1296
Epoch 60 / 100, Loss: 12.3298
Epoch 70 / 100, Loss: 10.6901
Epoch 80 / 100, Loss: 10.3057
Epoch 90 / 100, Loss: 10.1002
 - Metrics: Accuracy=0.8806, F1=0.8466, Recall=0.8248, Precision=0.8696
Running experiment with seed=114:
 - K=9, layers=1, hidden=64, out=128
 - norm=None, dropout=0, batch_size=256, methodology=ours
 - ratio=0.3197108882381444, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=cluster, sampling_k=25
 - model_type=GCNConv, rate_pairs=1, clusters=50, lr=0.0008492447031723229


Computing METIS partitioning...
Done!


...

 - Metrics: Accuracy=0.8877, F1=0.8528, Recall=0.8142, Precision=0.8952
Done. Results written to pubmed_experimentations\pubmed_scar_sampling_nnif_batch_cluster_0204110252.csv.
Average F1 over valid seeds: 0.8629 ± 0.0065
Running experiment with seed=654:
 - K=12, layers=2, hidden=128, out=128
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.2767860708256532, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=nearest_neighbors, sampling_k=25
 - model_type=GATConv, rate_pairs=2, clusters=200, lr=0.006805715622367233
Epoch 0 / 100, Loss: 97.3657
Epoch 10 / 100, Loss: 21.2856
Epoch 20 / 100, Loss: 7.4298
Epoch 30 / 100, Loss: 3.8046
Epoch 40 / 100, Loss: 2.3014
Epoch 50 / 100, Loss: 1.5345
Epoch 60 / 100, Loss: 1.0812
Epoch 70 / 100, Loss: 0.7949
Epoch 80 / 100, Loss: 0.5977
Epoch 90 / 100, Loss: 0.4590
 - Metrics: Accuracy=0.8955, F1=0.8665, Recall=0.8495, Precision=0.8842
Running experiment with seed=114:
 - K=12, layers=2, hidden=128, 

[I 2025-04-02 14:43:37,687] Trial 38 finished with value: 0.8696181994140113 and parameters: {'K': 12, 'layers': 2, 'hidden_channels': 128, 'out_channels': 128, 'ratio': 0.2767860708256532, 'model_type': 'GATConv', 'batch_size': 512, 'lr': 0.006805715622367233, 'clusters': 200, 'sampling': 'nearest_neighbors', 'sampling_k': 25}. Best is trial 17 with value: 0.8728735406239962.


 - Metrics: Accuracy=0.8994, F1=0.8722, Recall=0.8589, Precision=0.8858
Done. Results written to pubmed_experimentations\pubmed_scar_sampling_nnif_batch_cluster_0204113251.csv.
Average F1 over valid seeds: 0.8696 ± 0.0024
Running experiment with seed=654:
 - K=11, layers=2, hidden=256, out=128
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.3439080911116069, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=75
 - model_type=GCNConv, rate_pairs=2, clusters=50, lr=0.0002338337376611519
Epoch 0 / 100, Loss: 69.5158
Epoch 10 / 100, Loss: 67.0257
Epoch 20 / 100, Loss: 64.3459
Epoch 30 / 100, Loss: 62.8243
Epoch 40 / 100, Loss: 60.1029
Epoch 50 / 100, Loss: 57.8925
Epoch 60 / 100, Loss: 55.6815
Epoch 70 / 100, Loss: 53.6635
Epoch 80 / 100, Loss: 51.2176
Epoch 90 / 100, Loss: 48.8145
 - Metrics: Accuracy=0.8878, F1=0.8597, Recall=0.8603, Precision=0.8590
Running experiment with seed=114:
 - K=11, layers=2, hidden=256, o

[I 2025-04-02 15:20:47,277] Trial 39 finished with value: 0.8604007874978535 and parameters: {'K': 11, 'layers': 2, 'hidden_channels': 256, 'out_channels': 128, 'ratio': 0.3439080911116069, 'model_type': 'GCNConv', 'batch_size': 512, 'lr': 0.0002338337376611519, 'clusters': 50, 'sampling': 'neighbor', 'sampling_k': 75}. Best is trial 17 with value: 0.8728735406239962.


 - Metrics: Accuracy=0.8927, F1=0.8651, Recall=0.8615, Precision=0.8689
Done. Results written to pubmed_experimentations\pubmed_scar_sampling_nnif_batch_cluster_0204144337.csv.
Average F1 over valid seeds: 0.8604 ± 0.0025
Running experiment with seed=654:
 - K=9, layers=2, hidden=64, out=64
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.29748906944073894, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=nearest_neighbors, sampling_k=75
 - model_type=GATConv, rate_pairs=2, clusters=100, lr=0.00827560373997299
Epoch 0 / 100, Loss: 66.1340
Epoch 10 / 100, Loss: 9.5423


### Hyperparameter Optimization Pubmed
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "pubmed",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 2, 12),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.2, 0.4),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("layers", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [15,256,512,1024]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "pubmed_sar_sampling_nnif_batch_cluster.csv",
        "min":0.84,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":trial.suggest_categorical("num_epochs", [50,100,150]),         
        "sampling":trial.suggest_categorical("sampling",["neighbor","cluster","nearest_neighbors"]),         
        "sampling_k":trial.suggest_categorical("sampling_k",[25,50,75]),
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-24 05:51:18,349] A new study created in memory with name: no-name-7363eccd-5b06-40dc-87b0-25014df8c5a1


Running experiment with seed=654:
 - K=7, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.3362908893186948, aggregation=mean, treatment=removal, anomaly_detector=unweighted, sampling=cluster, sampling_k=50
 - model_type=GATConv, rate_pairs=10, clusters=300, lr=0.0007087214622711031


Computing METIS partitioning...
Done!


Epoch 0 / 100, Loss: 18.5522
Epoch 10 / 100, Loss: 17.8089
Epoch 20 / 100, Loss: 17.3418
Epoch 30 / 100, Loss: 16.8934
Epoch 40 / 100, Loss: 16.1184
Epoch 50 / 100, Loss: 15.1700
Epoch 60 / 100, Loss: 14.4180
Epoch 70 / 100, Loss: 13.4842
Epoch 80 / 100, Loss: 12.6036
Epoch 90 / 100, Loss: 11.9384


[I 2025-03-24 05:53:00,820] Trial 0 finished with value: 0.715743932941256 and parameters: {'K': 7, 'ratio': 0.3362908893186948, 'model_type': 'GATConv', 'batch_size': 1024, 'lr': 0.0007087214622711031, 'anomaly_detector': 'unweighted', 'clusters': 300, 'num_epochs': 100, 'sampling': 'cluster', 'sampling_k': 50}. Best is trial 0 with value: 0.715743932941256.


...

 - Metrics: Accuracy=0.8745, F1=0.8391, Recall=0.8193, Precision=0.8598
F1 = 0.84 < 0.84, skipping ...
Done. Results written to pubmed_experimentations\pubmed_sar_sampling_nnif_batch_cluster_2403133828.csv.
Average F1 over valid seeds: 0.8391 ± 0.0000
Running experiment with seed=654:
 - K=7, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.28001597838526016, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=nearest_neighbors, sampling_k=50
 - model_type=GCNConv, rate_pairs=10, clusters=400, lr=0.00873922908266354
Epoch 0 / 50, Loss: 58.0806
Epoch 10 / 50, Loss: 8.0599
Epoch 20 / 50, Loss: 2.8407
Epoch 30 / 50, Loss: 1.4709
Epoch 40 / 50, Loss: 0.8989
 - Metrics: Accuracy=0.8762, F1=0.8413, Recall=0.8212, Precision=0.8624
Running experiment with seed=114:
 - K=7, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.28001597838526016, aggregation=mean, treatmen

[I 2025-03-24 15:01:39,121] Trial 13 finished with value: 0.835498509343777 and parameters: {'K': 7, 'ratio': 0.28001597838526016, 'model_type': 'GCNConv', 'batch_size': 512, 'lr': 0.00873922908266354, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 50, 'sampling': 'nearest_neighbors', 'sampling_k': 50}. Best is trial 10 with value: 0.8395798210348853.


 - Metrics: Accuracy=0.8702, F1=0.8297, Recall=0.7916, Precision=0.8716
F1 = 0.83 < 0.84, skipping ...
Done. Results written to pubmed_experimentations\pubmed_sar_sampling_nnif_batch_cluster_2403141459.csv.
Average F1 over valid seeds: 0.8355 ± 0.0058
Running experiment with seed=654:
 - K=7, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.26308150498044675, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=nearest_neighbors, sampling_k=50
 - model_type=GCNConv, rate_pairs=10, clusters=400, lr=0.009817622280543212
Epoch 0 / 50, Loss: 57.2382
Epoch 10 / 50, Loss: 6.7150
Epoch 20 / 50, Loss: 2.3497


[W 2025-03-24 15:11:12,619] Trial 14 failed with parameters: {'K': 7, 'ratio': 0.26308150498044675, 'model_type': 'GCNConv', 'batch_size': 512, 'lr': 0.009817622280543212, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 50, 'sampling': 'nearest_neighbors', 'sampling_k': 50} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\romai\AppData\Local\Temp\ipykernel_22248\4249684348.py", line 39, in objective
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 565, in run_nnif_gnn_experiment
    train_labels, train_proba, train_losses = train_graph(
                                              ^^^^^^^^^^^

KeyboardInterrupt: 

### Hyperparameter Optimization WikiCS
#### SCAR 

In [5]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "wiki-cs",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 1, 35),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.09, 0.35),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("rate_pairs",1,15),
        "batch_size": trial.suggest_categorical("batch_size", [15,256,512,1024,15]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "wikics_scar_sampling_nnif_batch_cluster.csv",
        "min":0.88,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,      
        "sampling":trial.suggest_categorical("sampling",["neighbor","cluster","nearest_neighbors"]),         
        "sampling_k":trial.suggest_categorical("sampling_k",[25,50,75]),
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-28 04:20:38,026] A new study created in memory with name: no-name-b7804c88-d9f4-4272-be1e-f766b03ffaed
Computing METIS partitioning...


Running experiment with seed=654:
 - K=11, layers=1, hidden=128, out=64
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.1926695943076433, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=cluster, sampling_k=75
 - model_type=GCNConv, rate_pairs=15, clusters=100, lr=0.0025061759908526224


Done!


Epoch 0 / 100, Loss: 18.5554
Epoch 10 / 100, Loss: 17.3968
Epoch 20 / 100, Loss: 14.5902
Epoch 30 / 100, Loss: 13.5468
Epoch 40 / 100, Loss: 11.3363
Epoch 50 / 100, Loss: 10.3032
Epoch 60 / 100, Loss: 9.2580
Epoch 70 / 100, Loss: 8.0010
Epoch 80 / 100, Loss: 6.8279
Epoch 90 / 100, Loss: 5.9414
 - Metrics: Accuracy=0.9435, F1=0.8836, Recall=0.9369, Precision=0.8361
Running experiment with seed=114:
 - K=11, layers=1, hidden=128, out=64
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.1926695943076433, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=cluster, sampling_k=75
 - model_type=GCNConv, rate_pairs=15, clusters=100, lr=0.0025061759908526224


Computing METIS partitioning...
Done!


...

Epoch 0 / 100, Loss: 3.5138
Epoch 10 / 100, Loss: 2.8710
Epoch 20 / 100, Loss: 2.4437
Epoch 30 / 100, Loss: 2.1170
Epoch 40 / 100, Loss: 1.7531
Epoch 50 / 100, Loss: 1.5131
Epoch 60 / 100, Loss: 1.2861
Epoch 70 / 100, Loss: 1.1255
Epoch 80 / 100, Loss: 0.9426
Epoch 90 / 100, Loss: 0.8364
 - Metrics: Accuracy=0.9606, F1=0.9140, Recall=0.9149, Precision=0.9132
Running experiment with seed=281:
 - K=5, layers=2, hidden=256, out=128
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.15148200168671186, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=cluster, sampling_k=75
 - model_type=GCNConv, rate_pairs=14, clusters=50, lr=0.008372583695812601


Computing METIS partitioning...
Done!


Epoch 0 / 100, Loss: 3.2542
Epoch 10 / 100, Loss: 3.0165
Epoch 20 / 100, Loss: 2.9101
Epoch 30 / 100, Loss: 2.1438
Epoch 40 / 100, Loss: 1.7470
Epoch 50 / 100, Loss: 1.5032
Epoch 60 / 100, Loss: 1.2463
Epoch 70 / 100, Loss: 1.2354
Epoch 80 / 100, Loss: 0.9383
Epoch 90 / 100, Loss: 0.8335


[I 2025-03-29 19:17:01,312] Trial 199 finished with value: 0.9171701172307115 and parameters: {'K': 5, 'layers': 2, 'hidden_channels': 256, 'out_channels': 128, 'ratio': 0.15148200168671186, 'model_type': 'GCNConv', 'rate_pairs': 14, 'batch_size': 1024, 'lr': 0.008372583695812601, 'clusters': 50, 'sampling': 'cluster', 'sampling_k': 75}. Best is trial 137 with value: 0.9189642012571827.


 - Metrics: Accuracy=0.9607, F1=0.9128, Recall=0.8988, Precision=0.9272
Done. Results written to wiki-cs_experimentations\wikics_scar_sampling_nnif_batch_cluster_2903191355.csv.
Average F1 over valid seeds: 0.9172 ± 0.0037
Best trial:
  Average F1: 0.9189642012571827
  Best parameters:
    K: 8
    layers: 2
    hidden_channels: 256
    out_channels: 128
    ratio: 0.16224551381027605
    model_type: GCNConv
    rate_pairs: 13
    batch_size: 512
    lr: 0.007865383500608529
    clusters: 50
    sampling: cluster
    sampling_k: 75


### Hyperparameter Optimization WikiCS
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "wiki-cs",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 3, 40),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.09, 0.18),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("rate_pairs",1,10),
        "batch_size": trial.suggest_categorical("batch_size", [256,512,1024]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "wikics_sar_sampling_nnif_batch_cluster.csv",
        "min":0.88,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,         
        "sampling":trial.suggest_categorical("sampling",["neighbor","cluster","nearest_neighbors"]),         
        "sampling_k":trial.suggest_categorical("sampling_k",[25,50,75]),
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


# Elliptic Bitcoin

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "elliptic-bitcoin",      
        "mechanism": "SAR2",
        "train_pct": 0.5,
        "alpha": 0.5,
        "K": trial.suggest_int("K", 2, 20),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": trial.suggest_float("dropout", 0, 0.4),
        "ratio": trial.suggest_float("ratio", 0.001, 0.03),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("rate_pairs",1,15),
        "batch_size": trial.suggest_categorical("batch_size", [15,256,512,1024]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 1,
        "output_csv": "elliptic_bitcoin_cluster.csv",
        "min":0.65,
        "clusters":trial.suggest_categorical("clusters", [100,500,1000]),          
        "num_epochs":100,#trial.suggest_categorical("num_epochs", [50,100,150]),         
        "sampling":trial.suggest_categorical("sampling",["neighbor","cluster"]),         
        "sampling_k":25,#trial.suggest_categorical("sampling_k",[25,50,75]),
        }

    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-24 20:02:21,721] A new study created in memory with name: no-name-9c84a830-1d32-4a81-aa10-5679b684cd4d


Running experiment with seed=654:
 - K=7, layers=2, hidden=256, out=64
 - norm=None, dropout=0.1983021209568396, batch_size=10, methodology=ours
 - ratio=0.01676656982996787, aggregation=mean, treatment=removal, anomaly_detector=unweighted, sampling=neighbor, sampling_k=50
 - model_type=GATConv, rate_pairs=9, clusters=1000, lr=0.007076470456182623


[I 2025-03-24 20:02:40,646] Trial 0 finished with value: 0.0 and parameters: {'K': 7, 'layers': 2, 'hidden_channels': 256, 'out_channels': 64, 'dropout': 0.1983021209568396, 'ratio': 0.01676656982996787, 'model_type': 'GATConv', 'rate_pairs': 9, 'batch_size': 10, 'lr': 0.007076470456182623, 'anomaly_detector': 'unweighted', 'clusters': 1000, 'num_epochs': 50, 'sampling': 'neighbor', 'sampling_k': 50}. Best is trial 0 with value: 0.0.


Error: Labels must be in {0,1}, got [0].
Done. Results written to elliptic-bitcoin_experimentations\elliptic_bitcoin_cluster_2403200221.csv.
Average F1 over valid seeds: 0.0000 ± 0.0000
Running experiment with seed=654:
 - K=3, layers=1, hidden=128, out=256
 - norm=None, dropout=0.01605654337952993, batch_size=20, methodology=ours
 - ratio=0.024315606511573024, aggregation=mean, treatment=removal, anomaly_detector=unweighted, sampling=nearest_neighbors, sampling_k=50
 - model_type=GATConv, rate_pairs=7, clusters=1000, lr=0.0089094751164754


[I 2025-03-25 01:09:57,013] Trial 1 finished with value: 0.0 and parameters: {'K': 3, 'layers': 1, 'hidden_channels': 128, 'out_channels': 256, 'dropout': 0.01605654337952993, 'ratio': 0.024315606511573024, 'model_type': 'GATConv', 'rate_pairs': 7, 'batch_size': 20, 'lr': 0.0089094751164754, 'anomaly_detector': 'unweighted', 'clusters': 1000, 'num_epochs': 150, 'sampling': 'nearest_neighbors', 'sampling_k': 50}. Best is trial 0 with value: 0.0.


...

Running experiment with seed=654:
 - K=16, layers=1, hidden=256, out=256
 - norm=None, dropout=0.06199985776750046, batch_size=10, methodology=ours
 - ratio=0.022437251981747064, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=50
 - model_type=GATConv, rate_pairs=9, clusters=500, lr=0.007917920503781244
Error: Labels must be in {0,1}, got [0].
Done. Results written to elliptic-bitcoin_experimentations\elliptic_bitcoin_cluster_2503010957.csv.
Average F1 over valid seeds: 0.0000 ± 0.0000


[I 2025-03-25 01:10:21,032] Trial 3 finished with value: 0.0 and parameters: {'K': 5, 'layers': 2, 'hidden_channels': 256, 'out_channels': 128, 'dropout': 0.03228429726090343, 'ratio': 0.008491919490729881, 'model_type': 'GCNConv', 'rate_pairs': 6, 'batch_size': 20, 'lr': 0.008387901223664842, 'anomaly_detector': 'unweighted', 'clusters': 500, 'num_epochs': 150, 'sampling': 'neighbor', 'sampling_k': 25}. Best is trial 0 with value: 0.0.


Running experiment with seed=654:
 - K=5, layers=2, hidden=256, out=128
 - norm=None, dropout=0.03228429726090343, batch_size=20, methodology=ours
 - ratio=0.008491919490729881, aggregation=mean, treatment=removal, anomaly_detector=unweighted, sampling=neighbor, sampling_k=25
 - model_type=GCNConv, rate_pairs=6, clusters=500, lr=0.008387901223664842
Error: Labels must be in {0,1}, got [0].
Done. Results written to elliptic-bitcoin_experimentations\elliptic_bitcoin_cluster_2503011009.csv.
Average F1 over valid seeds: 0.0000 ± 0.0000
Running experiment with seed=654:
 - K=17, layers=1, hidden=128, out=128
 - norm=None, dropout=0.2915360402823104, batch_size=20, methodology=ours
 - ratio=0.020473770474233224, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=cluster, sampling_k=50
 - model_type=GCNConv, rate_pairs=3, clusters=1000, lr=0.009469031504558686


Computing METIS partitioning...


: 